In [1]:
from QuantLib import *

In [4]:
depo_maturities = [Period(6,Months), Period(12, Months)]
bond_maturities = [Period(6, Months) for i in range(3, 21)]

In [5]:
depo_rates = [5.25, 5.5]
bond_rates = [5.7, 6.0, 6.25, 6.5, 6.75, 6.80, 7.00, 7.1,
                  7.15, 7.2, 7.3, 7.35, 7.4, 7.5, 7.6, 7.7, 7.8]

In [8]:
reference_date = Date(15,1,2015)
Settings.instance().evaluationDate = reference_date
calc_date = reference_date

In [10]:
calendar = UnitedStates()
business_convention = Unadjusted
day_count = Thirty360()
end_of_month = True
settlement_days = 0
face_amount = 100
coupon_frequency = Period(Semiannual)
settlement_days = 0

In [11]:
depo_helpers = [DepositRateHelper(QuoteHandle(SimpleQuote(r/100.0)),
                                 m, 
                                 settlement_days,
                                 calendar,
                                 business_convention,
                                 end_of_month,
                                 day_count) 
               for r, m in zip(depo_rates, depo_maturities)]

In [12]:
depo_helpers

[<QuantLib.QuantLib.DepositRateHelper; proxy of <Swig Object of type 'DepositRateHelperPtr *' at 0x110c69d80> >,
 <QuantLib.QuantLib.DepositRateHelper; proxy of <Swig Object of type 'DepositRateHelperPtr *' at 0x110c69e10> >]

In [29]:
bond_helpers = []
for r, m in zip(bond_rates, bond_maturities):
    termination_date = calc_date + m
    schedule = Schedule(calc_date,
                          termination_date,
                          coupon_frequency,
                          calendar,
                          business_convention,
                          business_convention,
                          DateGeneration.Backward, 
                          end_of_month)
    helper = FixedRateBondHelper(QuoteHandle(SimpleQuote(face_amount)),
                                    settlement_days, 
                                    face_amount, 
                                    schedule, 
                                    [r/100.0],
                                    day_count,
                                    business_convention)
    bond_helpers.append(helper)

In [30]:
rate_helpers = depo_helpers + bond_helpers
yieldcurve = PiecewiseLogCubicDiscount(calc_date,
                                      rate_helpers,
                                      day_count)

### LIBOR CONSTRUCTION

In [33]:
helpers = [SwapRateHelper(QuoteHandle(SimpleQuote(rate/100.0)),
                          Period(*tenor), TARGET(),
                          Annual, Unadjusted,
                          Thirty360(),
                          Euribor6M())
          for tenor, rate in [((2, Years), 0.201),
                             ((3, Years), 0.258),
                             ((5, Years), 0.464),
                             ((10, Years), 1.151),
                             ((15, Years), 1.588)]]

In [34]:
curve1 = PiecewiseFlatForward(0, TARGET(), helpers, Actual360())

In [35]:
curve1.nodes()

((Date(15,1,2015), 0.001977769487929328),
 (Date(19,1,2017), 0.001977769487929328),
 (Date(19,1,2018), 0.0036675383554942956),
 (Date(20,1,2020), 0.007629248495439154),
 (Date(20,1,2025), 0.0184154249894188),
 (Date(21,1,2030), 0.025313247068010428))

In [38]:
dates, rate = zip(*curve1.nodes())

In [39]:
curve2 = ForwardCurve(dates, rate, Actual360())

In [40]:
print(curve2.referenceDate(), 'to', curve2.maxDate())

January 15th, 2015 to January 21st, 2030


In [ ]:
print